``Mission : Error Analysis & Model metadata``
- One great heuristic is all I need now!

# Data and dependencies

In [51]:
import warnings
import sys
import os
import time
import joblib
import random
from tqdm import tqdm
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder

from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

warnings.filterwarnings('ignore')

In [52]:
metadata = pd.read_csv('../data/metadata.csv')
sub_format = pd.read_csv('../data/submission_format.csv')
train_labels = pd.read_csv('../data/train_labels.csv')

# Add date fts

In [53]:
metadata.date = pd.to_datetime(metadata.date)
metadata['year'] = metadata.date.dt.year
metadata['month'] = metadata.date.dt.month
metadata['week'] = metadata.date.dt.isocalendar().week


seasons = {
    1: 1,
    2: 1,
    3: 2,
    4: 2,
    5: 2,
    6: 3,
    7: 3,
    8: 3,
    9: 4,
    10: 4,
    11: 4,
    12: 1
}

metadata['season'] = metadata.month.map(seasons)


data = pd.merge(metadata, train_labels, on='uid', how='left')
data.sort_values(by=['date'], inplace=True)
data

,uid,latitude,longitude,date,split,year,month,week,season,region,severity,density
4387,evep,44.847993,-93.476318,2013-01-04,train,2013,1,1,1,midwest,1.0,115.0
13644,paev,44.822478,-93.367962,2013-01-04,train,2013,1,1,1,midwest,1.0,1884.0
5566,gdxr,44.877646,-93.557842,2013-01-04,train,2013,1,1,1,midwest,1.0,1416.0
6144,guny,44.878889,-93.490833,2013-01-04,train,2013,1,1,1,midwest,1.0,558.0
5317,fwbt,44.850500,-93.515700,2013-01-04,train,2013,1,1,1,midwest,1.0,476.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12443,nsoi,36.736800,-121.734000,2021-12-29,test,2021,12,52,1,NaN,NaN,NaN
17559,thki,36.725400,-121.730000,2021-12-29,test,2021,12,52,1,NaN,NaN,NaN
17452,teuu,36.772300,-121.788000,2021-12-29,test,2021,12,52,1,NaN,NaN,NaN
14254,prfi,36.751800,-121.742000,2021-12-29,test,2021,12,52,1,NaN,NaN,NaN


In [54]:
train_data = data[data.split == 'train']
train_data.shape, data.shape

((17060, 12), (23570, 12))

In [56]:
grp_by_rw = data.groupby(['region', 'week']).severity.expanding(1).mean()
grp_by_rw = grp_by_rw.map(np.round)
print(grp_by_rw.isna().sum())
mse(train_data.severity.sort_index(), grp_by_rw.droplevel(0).droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0


0.8209593221806205

In [28]:
xgbreg_preds = pd.read_csv('../submissions/xgbreg_cvpreds_on_expanding_sev_rs_preds.csv')
xgbreg_preds.severity.value_counts()

2    2672
4    2029
1     987
3     820
5       2
Name: severity, dtype: int64

In [29]:
xgbreg_preds.severity.value_counts(normalize=True)

2    0.410445
4    0.311674
1    0.151613
3    0.125960
5    0.000307
Name: severity, dtype: float64

In [39]:
newxgbreg_preds = xgbreg_preds.copy()
newxgbreg_preds.severity = newxgbreg_preds.severity.map({
                                                        2:  1,
                                                        1 : 2,
                                                        3 : 3,
                                                        4 : 4,
                                                        5 : 5,
                                                    })

newxgbreg_preds.severity.value_counts(normalize=True)

1    0.410445
4    0.311674
2    0.151613
3    0.125960
5    0.000307
Name: severity, dtype: float64

In [41]:
newxgbreg_preds.to_csv('../submissions/modified_xgbreg_cvpreds_on_expanding_sev_rs_preds.csv', index=False)
#  idk what to expect, modified 1s and 2s to match ideal distribution of the data and see if it improves the score
#  but definetly decreases the errror but instead got a worse score @ 1.05

In [45]:
mse(train_labels.severity.values, [3]*train_labels.shape[0], squared=False)

1.472809439001117

In [49]:
1.0018 - 0.9777

0.02410000000000001

In [20]:
grp_by_yr = train_data.groupby('year').severity.expanding(1).mean()
grp_by_yr = grp_by_yr.map(np.round)
grp_by_yr.isna().sum()
mse(train_data.severity.sort_index(), grp_by_yr.droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

1.2071009130567592